# Database Joins 1: Combining JIRA With GitHub

For this notebook, our research question is as follows:

<b style="color:green">Are there Liferay Portal Patches (LPP) tickets that are currently stuck in review?</b>

In order to answer this question, this notebook introduces database joins by creating a script that combines data retrieved from JIRA with data retrieved from GitHub.

## Prerequisites

The following cell attempts to use `conda` and `pip` to install the libraries that are used by this notebook. If the output indicates that additional items were installed, you will need to restart the kernel after the installation completes before you can run the later cells in the notebook.

In [ ]:
!conda install -y numpy pandas pytz requests ujson
!pip install dateparser

## Notebook Imports

In [ ]:
from __future__ import print_function

from collections import defaultdict, namedtuple
import dateparser
from datetime import date, datetime
import hashlib
import numpy as np
import os
import pandas as pd
import pytz
import re
import requests
import six
import sys
import subprocess
import ujson as json

## Save Raw Data

Before we start, we'll make sure that we establish one rule for this script and all future scripts: we will save all raw data.

This is important, because one of the more common things to do as a developer is to retrieve the data, extract only the information you need, and then discard the data you do not need. However, unless you have some terms of service agreement restricting what data you are allowed to retain, do not discard the raw data! Raw data is a starting point that speeds up the creation of many different application prototypes.

In [ ]:
today = date.today()
now = datetime.now(pytz.utc)

def load_raw_data(cache_name):
    base_name = os.path.basename(cache_name)
    subfolder_name = os.path.dirname(cache_name)
    folder_name = 'rawdata/%s' % subfolder_name

    if not os.path.exists(folder_name):
        return None

    file_name = '%s/%s_%s.json' % (folder_name, today.isoformat(), base_name)

    if not os.path.isfile(file_name):
        return None

    with open(file_name) as infile:
        return json.load(infile)

def save_raw_data(cache_name, json_value):
    base_name = os.path.basename(cache_name)
    subfolder_name = os.path.dirname(cache_name)
    folder_name = 'rawdata/%s' % subfolder_name

    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    file_name = '%s/%s_%s.json' % (folder_name, today.isoformat(), base_name)

    with open(file_name, 'w') as outfile:
        json.dump(json_value, outfile)

## Load Data from JIRA, Part 1

Let's keep our question in mind during each step.

<b style="color:green">Are there Liferay Portal Patches (LPP) tickets that are currently stuck in review?</b>

Before we can answer the question of whether anything is *stuck in review*, we will need to answer the broader question of what is *in review*. We'll start by looking at what's in review in JIRA.

### Login to JIRA

Because [issues.liferay.com](https://issues.liferay.com/) does not have OAuth support, we will need to find a different way to connect to our JIRA installation. The simplest way is to simply login to JIRA.

There are a lot of secure ways to specify your username and password, but for the sake of this script, we'll use the most insecure way possible: a plain text file. Namely, the `.gitconfig` in your user's home folder. If you want to use a different strategy that's less global (a plain text JSON file in the same folder as this script, for example) or more secure, just change the implementation of the two functions below.

In [ ]:
def get_config(key):
    try:
        return subprocess.check_output(['git', 'config', key]).strip().decode('utf8')
    except:
        return None

def set_config(key, value):
    subprocess.call(['git', 'config', '--global', key, value])
    subprocess.call(['git', 'config', '--global', key, value])

Assuming you are using the default implementation, set your username and password inside of the `.gitconfig` located in your user's home folder by running the following two commands in a command line window.

``` .sh
git config --global jira.session-username JIRA_USERNAME
git config --global jira.session-password JIRA_PASSWORD
```

The following cell will read it in and confirm that it exists.

In [ ]:
jira_username = get_config('jira.session-username')
jira_password = get_config('jira.session-password')

assert(jira_username is not None)
assert(jira_password is not None)

The following cell will use this information and attempt to login to JIRA and confirm that it has a valid session, which will confirm that the credentials you saved are valid. It will also save this session information so that it can reuse it later without constantly relogging in.

In [ ]:
jira_base_url = 'https://issues.liferay.com/rest'

def get_jira_cookie():
    jira_cookie = None

    jira_cookie_name = None
    jira_cookie_value = None

    try:
        jira_cookie_name = get_config('jira.session-cookie-name')
        jira_cookie_value = get_config('jira.session-cookie-value')
    except:
        pass
    
    if jira_cookie_name is not None and jira_cookie_value is not None:
        jira_cookie = {
            jira_cookie_name: jira_cookie_value
        }

        r = requests.get(jira_base_url + '/auth/1/session', cookies=jira_cookie)

        if r.status_code != 200:
            jira_cookie = None

    if jira_cookie is not None:
        return jira_cookie
        
    post_json = {
        'username': jira_username,
        'password': jira_password
    }

    r = requests.post(jira_base_url + '/auth/1/session', json=post_json)

    if r.status_code != 200:
        print('Invalid login')

        return None

    response_json = r.json()

    jira_cookie_name = response_json['session']['name']
    jira_cookie_value = response_json['session']['value']

    set_config('jira.session-cookie-name', jira_cookie_name)
    set_config('jira.session-cookie-value', jira_cookie_value)

    jira_cookie = {
        jira_cookie_name: jira_cookie_value
    }

    return jira_cookie

In [ ]:
assert(get_jira_cookie() is not None)

### Retrieve JIRA Issues

Now that we have a valid login, our next step is to use the JIRA API to retrieve tickets. If you've interacted with JIRA before, you know that it has its own query language (JQL). It turns out there is a simple search API that allows you to submit the JQL and all the matching issues are returned as JSON. Since the API is fairly simple, we implement it here.

In [ ]:
def retrieve_jira_issues(jql):
    jira_cookie = get_jira_cookie()

    if jira_cookie is None:
        return []

    start_at = 0

    post_json = {
        'jql': jql,
        'startAt': start_at
    }

    r = requests.post(jira_base_url + '/api/2/search', cookies=jira_cookie, json=post_json)

    if r.status_code != 200:
        return []

    response_json = r.json()

    issues = response_json['issues']

    while start_at + response_json['maxResults'] < response_json['total']:
        start_at += response_json['maxResults']
        post_json['startAt'] = start_at

        r = requests.post(jira_base_url + '/api/2/search', cookies=jira_cookie, json=post_json)

        if r.status_code != 200:
            return issues

        response_json = r.json()

        issues.extend(response_json['issues'])

    return issues

Now that we have something that can retrieve JIRA issues, all we need is to actually create our JQL and then run the search. This is the JQL we'll use for regular Liferay Portal Patch (LPP) issues that are in review.

In [ ]:
in_review_jql = '''
    project = LPP AND
    type not in ("SME Request", "SME Request SubTask") AND
    status = "In Review"
    order by key
'''

Let's go ahead and retrieve those results. Looking at JSON is a bit tedious, so we'll take a look at a subset of fields in a way that resembles the view you get when you run JQL via the web browser.

In [ ]:
jql_hashes = load_raw_data('jql_hashes')

if jql_hashes is None:
    jql_hashes = {}

def get_jql_hashed_name(base_name, jql):
    jql_hash = None
    
    for key, value in jql_hashes.items():
        if value == jql:
            jql_hash = key
            break

    if jql_hash is None:            
        digester = hashlib.md5()
        digester.update(jql)
        jql_hash = digester.hexdigest()

        jql_hashes[jql_hash] = jql

        save_raw_data('jql_hashes', jql_hashes)
    
    return '%s/%s' % (jql_hash, base_name)

def get_jira_issues(jql):
    base_name = get_jql_hashed_name('jira_issues', jql)

    jira_issues = load_raw_data(base_name)

    if jira_issues is not None:
        print('Loaded cached JIRA search')
        return jira_issues

    print('Executing JIRA search')

    jira_issues = retrieve_jira_issues(in_review_jql)
    save_raw_data(base_name, jira_issues)
    return jira_issues

In [ ]:
if __name__ == '__main__':
    jira_issues = get_jira_issues(in_review_jql)
else:
    jira_issues = []

In [ ]:
JIRAIssue = namedtuple(
    'JIRAIssue',
    ['key', 'region', 'status', 'assignee', 'summary']
)

def get_jira_tuple(issue):
    region_field_name = 'customfield_11523'

    regions = ['']

    if region_field_name in issue['fields']:
        regions = [region['value'] for region in issue['fields'][region_field_name]]

    return JIRAIssue(
        key=issue['key'],
        region=regions[0],
        status=issue['fields']['status']['name'],
        assignee=issue['fields']['assignee']['displayName'],
        summary=issue['fields']['summary']
    )

In [ ]:
pd.DataFrame([get_jira_tuple(item) for item in jira_issues])

## Detour: Derived Tables

As seen in the result of executing the previous code cell, the result of our API call against JIRA using JQL can be conceptualized as a table.

Just as lists of results can be conceptualized as a table, so too are things that work with regular SQL. More explicitly, whenever you're working with a database and you execute SQL, the result can be understood to be another table, whether this table consists of a single value, a single row, or multiple rows.

* [Relational algebra](https://en.wikipedia.org/wiki/Relational_algebra)

At a conceptual level, running database queries is taking existing tables and generating (or deriving) new tables that better fit the question you are trying to answer.

### Persisting Derived Tables

The last point is worth emphasizing, as it comes up in upgrade performance tuning. In short, running database queries always generates new tables. More explicitly, the database will have gone through all of the effort required to create a table when executing your query, skipping only the persistence step.

* [The difference between subqueries and derived tables in SQL](https://www.xaprb.com/blog/2005/09/26/sql-subqueries-and-derived-tables/)

Looking at this a little differently, imagine that we decided to remove the code that invoked the `load_raw_data` and `save_raw_data` functions. Would this have noticeably improved the performance of our code? Not really, because the retrieval from JIRA is slower than writing the data to disk. That's equivalent to the cost difference between persisting the data and not persisting it.

Having the derived table be saved as an permanent table also allows you to add metadata, such as indices, that will improve the performance on repeated queries. So, in cases where you are operating on different subsets of a larger subset of the data, and this larger subset is expensive to compute, it's actually very sensible to save your derived table to reduce query execution time.

## Load Data from GitHub, Part 1

Let's remind ourselves of our original question.

<b style="color:green">Are there Liferay Portal Patches (LPP) tickets that are currently stuck in review?</b>

What is this concept of "in review"? Well, being "in review" actually means that your code has been written and you are now waiting on another team member to look at your changes, providing a sanity check from someone who is looking at the solution with fresh eyes.

Where do these sanity check reviews occur? They occur on GitHub. This means that we will want to bring in the GitHub data set in order to answer our question.

### Login to GitHub

Luckily, [api.github.com](https://developer.github.com/v3/) does have OAuth support, so we'll want to request an OAuth token from GitHub and leverage it in our script.

* [Creating a personal access token for command line](https://help.github.com/articles/creating-a-personal-access-token-for-the-command-line/)

Assuming you are using the default implementation for configuration values provided by this notebook (mentioned earlier when setting up JIRA access), set your OAuth token inside of the `.gitconfig` located in your user's home folder by running the following command in a command line window.

``` .sh
git config --global github.oauth-token GITHUB_OAUTH_TOKEN
```

If you customized it, do whatever you need to get the configuration value persisted. The following cell will read it in and confirm that it exists.

In [ ]:
github_oauth_token = get_config('github.oauth-token')

assert(github_oauth_token is not None)

However, it's not enough that it exists. We should confirm that the token works on the repositories that are related to Liferay. We'll use the `liferay/liferay-portal` and `liferay/liferay-portal-ee` repositories as a way to validate.

In [ ]:
github_base_url = 'https://api.github.com'

def is_repository_accessible(reviewer_url):
    print('Validating OAuth token against %s' % reviewer_url)

    headers = {
        'user-agent': 'python checklpp.py',
        'authorization': 'token %s' % github_oauth_token
    }

    api_path = '/repos/%s' % reviewer_url
    
    r = requests.get(github_base_url + api_path, headers=headers)
    
    return r.status_code == 200

assert(is_repository_accessible('liferay/liferay-portal'))
assert(is_repository_accessible('liferay/liferay-portal-ee'))

## Detour: Join Strategies

We currently have one data set (a list of JIRA tickets that are in review), and our next step is to think about how we can answer our actual question.

<b style="color:green">Are there Liferay Portal Patches (LPP) tickets that are currently stuck in review?</b>

Now that we've identified which LPP tickets are currently in review, our next step is to identify pull requests that are stuck in review. So, that leaves us with the following question: how should we implement this?

### Goals of Query Optimization

What will happen from here is you ask the database to provide an explanation of what it's doing for your query, you work some magic to make this explanation look better (add database indices, modify the query), and you commit those changes to the codebase.

* [Query plan](https://en.wikipedia.org/wiki/Query_plan)

Our definition of "better" comes from interpreting certain aspects of the query plan as necessarily worse than what we expected. Our expectations often come from definitions that are provided by various database vendors on the ideal query plan.

* [MySQL explain plan](https://dev.mysql.com/doc/refman/5.6/en/explain-output.html)
* [Oracle explain plan](https://docs.oracle.com/database/121/TGSQL/tgsql_interp.htm)
* [PostgreSQL explain plan](http://www.postgresql.org/docs/9.4/static/using-explain.html)
* [SQL Server explain plan](https://technet.microsoft.com/en-us/library/ms178071%28v=sql.105%29.aspx)

When you run into a slow query and you ask for it to provide an explanation of its approach, the output you receive is the query plan derived by the query optimizer.

* [Query optimization](https://en.wikipedia.org/wiki/Query_optimization)

Query optimization plans essentially evaluate various ways of correctly answering the query described by the SQL statement while also minimizing the cost of loading the query. For a database, this notion of "cost" can be summarized as use of memory (in particular, pages getting swapped in and out of active memory), disk I/O, and network I/O.

* [Relative time cost of computer operations](http://norvig.com/21-days.html#answers)

The result of this optimized plan particularly easy to understand in Microsoft SQL Server, where it gives you a graphical representation of its query plan, and it estimates what percentage of the time spent processing the query will be spent on that specific aspect of it.

* [SQL Server query execution plans](http://www.sqlshack.com/sql-server-query-execution-plans-understanding-reading-plans/)
* [SQL Server graphical plan icons](https://technet.microsoft.com/en-us/library/ms175913%28v=sql.105%29.aspx)

### Goals of Join Strategies

Taking a step back, what are we doing is that we need to derive new information from two separate data sources. If these two data sources were both tables, then if these tables have columns that refer to the same abstract concept (or perhaps a third "mapping table" that describes that abstract concept), then we would join these two 

* [Khan Academy: Joining Related Tables](https://www.khanacademy.org/computing/computer-programming/sql/relational-queries-in-sql/p/joining-related-tables)

We've identified all of the JIRA issues that are in review. Presumably, we want to then combine this with GitHub data. What algorithm will we implement to achieve this? It turns out there are three basic strategies for computing a join: a nested loop join, a hash join, and a sort-merge join.

* [Join methods and subqueries](http://www.orafaq.com/tuningguide/join%20methods.html)

Sort-merge deserves its own discussion (which we'll do in the next tutorial), and it will make more sense after we're finished than before we've finished. So we'll look first at how a nested loop join and a hash join relate to our problem.

### Nested Loop Join

A simple solution to this problem would be to iterate over each of our pull requests and then check each JIRA ticket to see if the JIRA ticket references the pull request. So for every GitHub pull request, you would check every JIRA ticket. This strategy is known as a **nested loop join**.

A nested loop join is a join where the query optimizer decides the best way to accomplish the join is to designate one table as the "outer table" (no connection to the notion of an outer join) and the other table as the "inner table", structured in much the same way as a for loop.

* [Nested loop join](https://en.wikipedia.org/wiki/Nested_loop_join)

How fast is a nested loop join? Let $m$ be the size of the JIRA table and $n$ be the size of the GitHub table. Regardless of which table you choose for the outer table, a nested loop join would have an expected runtime and a worst-case runtime of $O(m \cdot n)$.

### Hash Join

Another solution to this problem would be to first load all the pull requests into a lookup data structure. From there, iterate over the JIRA tickets and match the pull requests to the hash table. So for every JIRA ticket, you would perform a number of lookups against our data structure not based on either tables size (so, effectively a constant). This strategy is known as a **hash join**.

A hash join is a join where the query optimizer decides the best way to accomplish the join is to build a lookup table on the smaller table (because it's more likely that a smaller table can fit into memory), with a popular choice being a hash table, and then iterate over the larger table.

* [Hash join](https://en.wikipedia.org/wiki/Hash_join)

How fast is a hash join? Let $m$ be the size of the JIRA table and $n$ be the size of the GitHub table. Note that hash tables have an $O(1)$ expected access and insertion time, but an $O(n)$ worst case access and insertion time (due to hash collisions, table resizes). So in practice, we have an expected runtime that is $O(m) + O(n)$ and a worst-case runtime that is $O(m \cdot n) + O(n^2)$.

## Load Data from GitHub, Part 2

Now that we have a valid login, we can proceed with our question.

<b style="color:green">Are there Liferay Portal Patches (LPP) tickets that are currently stuck in review?</b>

As noted previously, before we can answer the question of whether anything is *stuck in review*, we will need to answer the broader question of what is *in review*. Now that we know what's in review in JIRA, we also want to what's in review on GitHub. Then, we will want to join these two tables together.

Between a nested loop join and a hash join, the hash join has the better expected runtime. So how can we perform a hash join? We need to create a lookup data structure containing all the pull requests using some key.

### Retrieve GitHub Pull Requests

Unfortunately, given the number of unique users repositories that are involved in reviewing Liferay pull requests, it's not practical to make an API call against every user and every repository. Instead, we'll want to do this on-demand based on which pull requests we know exist.

If you visit `/repos/USERNAME/REPOSITORY/pulls` without a pull ID, GitHub will return all currently open pull requests. The approach below uses this API in an attempt to reduce the number of API requests to GitHub, because this will fetch all pull requests that are open in a single request.

Following our principle of keeping any data we retrieve, we save all of these pull requests, and then we request any additional pull requests that are closed and save those as well. Because our end goal is a hash join, we'll load this table as a map or dictionary where the key is the GitHub URL, because that's what will be present in JIRA fields.

In [ ]:
def retrieve_pull_requests(reviewer_url, pull_request_ids=[]):
    print('Checking pull requests waiting on %s' % reviewer_url)

    headers = {
        'user-agent': 'python checklpp.py',
        'authorization': 'token %s' % github_oauth_token
    }

    api_path = '/repos/%s/pulls' % reviewer_url

    r = requests.get(github_base_url + api_path, headers=headers)

    if r.status_code != 200:
        return {}

    new_pull_requests = r.json()

    new_seen_pull_requests = { pull_request['html_url']: pull_request for pull_request in new_pull_requests }

    for pull_request_id in pull_request_ids:
        github_url = 'https://github.com/%s/pull/%s' % (reviewer_url, pull_request_id)

        if github_url in new_seen_pull_requests:
            continue

        api_path = '/repos/%s/pulls/%s' % (reviewer_url, pull_request_id)

        r = requests.get(github_base_url + api_path, headers=headers)

        if r.status_code != 200:
            continue

        new_seen_pull_requests[github_url] = r.json()

    return new_seen_pull_requests

For now, we'll take a look at all the pull requests open against `liferay/liferay-portal-ee`.

In [ ]:
def get_open_backports():
    open_backports = load_raw_data('open_backports')

    if open_backports is not None:
        print('Loaded cached open backports')
        return open_backports
        
    open_backports = retrieve_pull_requests('liferay/liferay-portal-ee')
    save_raw_data('open_backport_pulls', open_backports)
    return open_backports

In [ ]:
if __name__ == '__main__':
    open_backports = get_open_backports()
else:
    open_backport_pulls = []

In [ ]:
GHPullRequest = namedtuple(
    'GHPullRequest',
    ['submitter', 'pull_id', 'branch', 'created_at', 'updated_at', 'state', 'github_url']
)

def get_github_tuple(pull_request):
    pull_id = '%s/%s#%d' % (
        pull_request['base']['user']['login'],
        pull_request['base']['repo']['name'],
        pull_request['number']
    )
    
    return GHPullRequest(
        submitter=pull_request['user']['login'],
        pull_id=pull_id,
        branch=pull_request['base']['ref'],
        created_at=pull_request['created_at'],
        updated_at=pull_request['updated_at'],
        state=pull_request['state'],
        github_url=pull_request['url']
    )

In [ ]:
pd.DataFrame([get_github_tuple(pull_request) for pull_request in open_backports.values()])

## Detour: Database Indices

Database indices are much easier to understand at an implementation level after you understand sort-merge joins, but we're going to delay that discussion until we have some more background. Instead, we'll focus on the concept behind a database index.

* [Database index](https://en.wikipedia.org/wiki/Database_index)


```
TODO TODO TODO TODO TODO TODO TODO TODO
TODO TODO TODO TODO TODO TODO TODO TODO
TODO TODO TODO TODO TODO TODO TODO TODO
TODO TODO TODO TODO TODO TODO TODO TODO
```

## Load Data from JIRA, Part 2

Now that we have a lookup data structure for GitHub, we look back at our question.

<b style="color:green">Are there Liferay Portal Patches (LPP) tickets that are currently stuck in review?</b>

Well, the thing to remember here is that the database query can go either way. Just as you might want to lookup GitHub pull requests by their URL, you will also want to lookup JIRA tickets by their ticket key.

In [ ]:
jira_issues_by_key = {issue['key']: issue for issue in jira_issues}

## Detour: Mapping Tables

Now that we have a function that can retrieve GitHub pull request metadata, you could say that in addition to having a JIRA table and a GitHub table. However, how can we join the two together?

The answer is a strategy that you see implemented in Liferay Service Builder. This strategy is known as a mapping table, named for how it explicitly creates many-to-many mappings.

* [Understanding Mapping Tables](https://stackoverflow.com/questions/6453462/mysql-understanding-mapping-tables)

### Mapping Table Extensions

Let's assume you have two tables, JIRA and GitHub You can use a mapping table in order to document the primary keys of JIRA and how they relate to the primary keys on GitHub.

What kinds of question can you answer with a mapping table? Simplistically, you could fetch all JIRA tickets corresponding to a specific GitHub pull request and all GitHub pull requests corresponding to a specific JIRA ticket. If you perform a three-way join, you could get all JIRA tickets and all GitHub pull requests that reference each other.

To answer these questions, you have to think more carefully about how to optimize these queries. A simple solution that works on smaller data sets is to load the entire mapping table into memory and perform hash joins on both sides.

However, it turns out that you can do better. There is a logical extension of a mapping table that brings together more than two keys, and this extension is referred to as a star schema. These are actually common in data warehouses, and databases can be configured (usually through the use of specialized database index types) to drastically improve query cost plans against star schemas.

* [Star schema](https://en.wikipedia.org/wiki/Star_schema)

## Load Data from GitHub, Part 3

So now that we have our lookup data structure, what do we need to answer our question?

<b style="color:green">Are there Liferay Portal Patches (LPP) tickets that are currently stuck in review?</b>

Well, while we have a lookup data structure, it turns out we don't actually have a GitHub table yet, only a subset of the GitHub table corresponding to pull requests sent against `liferay/liferay-portal-ee`! So our next step is to generate the relevant portion of our GitHub table.

### Identify GitHub Pulls Related to JIRA Issues

So while we have a utility method that allows us to retrieve GitHub pull request histories, we don't actually have most of that information. So how do we identify which ones to retrieve?

Our first step is to iterate over each JIRA ticket and extract the pull requests contained in the ticket. Once we have that information, we'll be able to use our lookup data structure.

In [ ]:
def extract_jira_pull_request_urls(jira_issues):
    issues_by_request = defaultdict(set)
    requests_by_reviewer = defaultdict(set)

    for jira_issue in jira_issues:
        for value in jira_issue['fields'].values():
            if not isinstance(value, six.string_types):
                continue

            for github_url in re.findall('https://github.com/[^\s]*/pull/[\d]+', value):
                issues_by_request[github_url].add(jira_issue['key'])

                pos = github_url.find('/', github_url.find('/', 19) + 1)
                reviewer_url = github_url[19:pos]
                pull_request_id = github_url[github_url.rfind('/') + 1:]

                requests_by_reviewer[reviewer_url].add(pull_request_id)

    return issues_by_request, requests_by_reviewer

Now, let's pass the data the results of our previous JIRA search to this function.

In [ ]:
def get_jira_pull_request_urls(jql):
    base_name_1 = get_jql_hashed_name('issues_by_request', jql)
    base_name_2 = get_jql_hashed_name('requests_by_reviewer', jql)
    
    issues_by_request = load_raw_data(base_name_1)
    requests_by_reviewer = load_raw_data(base_name_2)

    if issues_by_request is not None and requests_by_reviewer is not None:
        print('Loaded cached JIRA to GitHub mapping')
        return issues_by_request, requests_by_reviewer
    
    jira_issues = get_jira_issues(jql)
    issues_by_request, requests_by_reviewer = extract_jira_pull_request_urls(jira_issues)

    save_raw_data(base_name_1, issues_by_request)
    save_raw_data(base_name_2, requests_by_reviewer)
    
    return issues_by_request, requests_by_reviewer

In [ ]:
if __name__ == '__main__':
    issues_by_request, requests_by_reviewer = get_jira_pull_request_urls(in_review_jql)
else:
    issues_by_request = {}
    requests_by_reviewer = {}

In [ ]:
JIRAGitHubMapping = namedtuple('JIRAGitHubMapping', ['jiraKey', 'githubKey'])

pd.DataFrame([
    JIRAGitHubMapping(jiraKey=jiraKey, githubKey=githubKey)
        for githubKey, jiraKeys in issues_by_request.items()
            for jiraKey in jiraKeys
])

As you can see, it turns out that what we've done is equivalent to building a mapping table between the JIRA tickets (represented with their issue key) and the GitHub pull requests (represented with their URL).

### Retrieve GitHub Pulls Related to JIRA Issues

Now, we'll want to fetch all the metadata associated with all those pull requests. We'll also want a quick way to separate the open/active pull requests from the closed/inactive pull requests so that we don't have to re-derive that metadata later on when we attempt to identify stuck pull requests.

In [ ]:
def retrieve_related_pull_requests(issues_by_request, requests_by_reviewer):
    seen_pull_requests = {}

    for reviewer_url, pull_request_ids in sorted(requests_by_reviewer.items()):
        new_seen_pull_requests = retrieve_pull_request_reviews(reviewer_url, pull_request_ids)
        seen_pull_requests.update(new_seen_pull_requests)

    return seen_pull_requests

Let's go ahead and use our mapping tables to populate the subset of the GitHub table corresponding to only (1) currently active pull requests, or (2) closed pull requests corresponding to a Liferay Portal Patch ticket that is currently in review.

In [ ]:
def get_related_pull_requests(jql):
    base_name = get_jql_hashed_name('seen_pull_requests', jql)
    
    seen_pull_requests = load_raw_data(base_name)

    if seen_pull_requests is not None:
        print('Loaded cached pull request metadata')
        return seen_pull_requests

    issues_by_request, requests_by_reviewer = extract_jira_pull_request_urls(jql)

    seen_pull_requests = retrieve_related_pull_requests(issues_by_request, requests_by_reviewer)
    save_raw_data(base_name, seen_pull_requests)
    
    return seen_pull_requests

In [ ]:
if __name__ == '__main__':
    seen_pull_requests = get_related_pull_requests(in_review_jql)
else:
    seen_pull_requests = {}

In [ ]:
pd.DataFrame([get_github_tuple(pull_request) for pull_request in seen_pull_requests.values()])

## Perform JIRA-GitHub Join

We now have raw data from both JIRA and GitHub, excluding comments. How can we proceed in answering our original question?

<b style="color:green">Are there Liferay Portal Patches (LPP) tickets that are currently stuck in review?</b>

Our next question will be, what pull requests are still open for each LPP ticket? From there, we'll then be able to look for tickets that are stuck.

### Open LPP Pull Requests

We proceed by identifying the open pull requests that are still open (scanning the left table and filtering). We can then use our mapping table

## Add Computed Fields

To answer this, now we need to now answer the following question: given a list of LPP tickets and given a list of pull requests, how do we identify the ones that are *stuck*? Well, that's something we can derive by checking how long pull requests have stayed open.

### Add GitHub Idle Time

In [ ]:
GHIdlePullRequest = namedtuple(
    'GHIdlePullRequest',
    list(GHPullRequest._fields) + ['open_time', 'idle_time']
)

def get_github_time_tuple(pull_request):
    old_tuple = get_github_tuple(pull_request)

    created_at = dateparser.parse(pull_request['created_at'])
    open_time = now - created_at

    updated_at = dateparser.parse(pull_request['updated_at'])
    idle_time = now - updated_at
    
    new_tuple = GHIdlePullRequest(
        open_time=open_time,
        idle_time=idle_time,
        **old_tuple._asdict()
    )

    return new_tuple

In [ ]:
pd.DataFrame([get_github_time_tuple(pull_request) for pull_request in seen_pull_requests.values()])

### Add JIRA Idle Time

## Add Group By Expression

## Generate the Report

In [ ]:
def get_daycount_string(time_delta):
    elapsed = float(time_delta.days) + float(time_delta.seconds) / (60 * 60 * 24)
    elapsed_string = '%0.1f days' % elapsed

    return elapsed_string

def report_active(outfile, jira_issues, issues_by_request, active_reviews, seen_pull_requests):
    jira_issues_by_key = { issue['key']: issue for issue in jira_issues }

    outfile.write('<h2>Active Pull Requests on %s</h2>' % today.isoformat())

    outfile.write('<table>')
    outfile.write('<tr>')

    for header in ['Submitter', 'Pull Request Link', 'Waiting Tickets', 'Open Time', 'Idle Time']:
        outfile.write('<th>%s</th>' % header)

    outfile.write('</tr>')

    for github_url in active_reviews:
        outfile.write('<tr>')

        if github_url not in seen_pull_requests:
            continue
        
        pull_request = seen_pull_requests[github_url]

        # Submitter

        outfile.write('<td>%s</td>' % pull_request['user']['login'])

        # Pull Request Link

        outfile.write('<td><a href="%s">%s#%d</a></td>' % (github_url, pull_request['base']['user']['login'], pull_request['number']))

        # Waiting Tickets

        affected_issue_keys = [issue_key for issue_key in issues_by_request[github_url]]
        affected_issue_urls = ['https://issues.liferay.com/browse/%s' % issue_key for issue_key in affected_issue_keys]
        affected_issue_assignees = [jira_issues_by_key[issue_key]['fields']['assignee']['displayName'] for issue_key in affected_issue_keys]

        affected_issue_links = [
            '<a href="%s">%s</a> (%s)' % (issue_url, issue_key, issue_assignee)
                for issue_key, issue_url, issue_assignee in zip(affected_issue_keys, affected_issue_urls, affected_issue_assignees)
        ]

        outfile.write('<td>%s</td>' % '<br />'.join(affected_issue_links))

        # Open Time

        created_at = dateparser.parse(pull_request['created_at'])
        open_time = now - created_at

        outfile.write('<td>%s</td>' % get_daycount_string(open_time))

        # Idle Time

        updated_at = dateparser.parse(pull_request['updated_at'])
        idle_time = now - updated_at

        outfile.write('<td>%s</td>' % get_daycount_string(idle_time))

        outfile.write('</tr>')

    outfile.write('</table>')

def report_completed(outfile, jira_issues, issues_by_request, active_reviews, seen_pull_requests):
    requests_by_issue = defaultdict(set)

    for github_url, issue_keys in issues_by_request.items():
        for issue_key in issue_keys:
            requests_by_issue[issue_key].add(github_url)

    completed_review = []
    region_field_name = 'customfield_11523'

    for issue in jira_issues:
        issue_key = issue['key']
        github_urls = requests_by_issue[issue_key]
        all_pulls_closed = len([github_url for github_url in github_urls if github_url in active_reviews]) == 0

        if not all_pulls_closed:
            continue

        assignee = issue['fields']['assignee']['displayName']

        regions = []

        if region_field_name in issue['fields']:
            regions = [region['value'] for region in issue['fields'][region_field_name]]

        region = ''

        if len(regions) > 0:
            region = regions[0]

        idle_time = None

        for github_url in github_urls:
            if github_url not in seen_pull_requests:
                continue
            
            pull_request = seen_pull_requests[github_url]

            if pull_request['closed_at'] is None:
                continue

            closed_at = dateparser.parse(pull_request['closed_at'])
            new_idle_time = now - closed_at

            if idle_time is None or new_idle_time < idle_time:
                idle_time = new_idle_time

        completed_review.append((region, issue_key, assignee, idle_time))

    completed_review.sort()

    if len(completed_review) > 0:
        outfile.write('<h2>Review Already Completed for %s</h2>' % today.isoformat())

        outfile.write('<table>')
        outfile.write('<tr>')

        for header in ['Region', 'Ticket', 'Idle Time']:
            outfile.write('<th>%s</th>' % header)

        for region, issue_key, assignee, idle_time in completed_review:
            outfile.write('<tr>')

            # Region

            outfile.write('<td>%s</td>' % region)

            # Ticket

            outfile.write('<td><a href="https://issues.liferay.com/browse/%s">%s</a> (%s)</td>' % (issue_key, issue_key, assignee))

            # Idle Time

            outfile.write('<td>%s</td>' % get_daycount_string(idle_time))

            outfile.write('</tr>')

        outfile.write('</table>')

In [ ]:
report_file_name = 'report_%s.html' % today.isoformat()

with open(report_file_name, 'w') as outfile:
    report_active(outfile, jira_issues, issues_by_request, active_reviews, seen_pull_requests)
    report_completed(outfile, jira_issues, issues_by_request, active_reviews, seen_pull_requests)

### Sort-Merge Join

If you are concerned about worst case runtimes, you could instead build a tree-like data structure that has $O(\log n)$ insertion and lookup time. As a result, the hash join would have a worst-case runtime of $O(m \log n) + O(n \log n)$.

Imagine that for every pull request, we've extracted its pull request URL and created a sorted data structure that allows us to lookup pull request metadata by its URL. Imagine also that we've extracted all the pull requests from every LPP ticket and we have a sorted data structure that allows us to lookup every JIRA issue corresponding to each GitHub URL.

A naive solution that uses these two data structures is a hash join. A smarter solution is to view this as the equivalent of merging two already sorted lists via merge sort.

* [Sorting visualizations](http://cs.stanford.edu/people/jcjohns/sorting.js/)

A sort-merge join is a join where the query optimizer decides the best way to accomplish the join is to sort the two tables on the specified join keys and then walk both tables in the same way as the merge step of merge sort.

* [Sort-merge join](https://en.wikipedia.org/wiki/Sort-merge_join)

Note that it doesn't necessarily have to use merge sort for this sort step, though merge sort is well-known to be one of the best external sorting algorithms (it's used by Hadoop during its shuffle step, for example).

* [External sorting](https://en.wikipedia.org/wiki/External_sorting)

From there, it will compare the sorted tables. It will place a cursor at the smallest key value for both tables (at the top of the sorted tables). At each step, it determines whether it should advance the cursor on one table or the other based on the values of the keys for the rows located at the current cursor position.

If you need a visualization in order to understand how the cursor advances in merge sort (possibly due to lack of familiarity with the merge sort algorithm), you're encouraged to consult this visualization.

* https://www.youtube.com/watch?v=kPRA0W1kECg#t=1m7

## Convert Notebook to Script

The following cell will use `jupyter nbconvert` to build an `checklpp.py` which runs the script outlined in this notebook.

In [ ]:
%%javascript
var script_file = 'checklpp.py';

var notebook_name = window.document.getElementById('notebook_name').innerHTML;
var nbconvert_command = 'jupyter nbconvert --stdout --to script ' + notebook_name;

var grep_command = "grep -v '^#' | grep -v -F get_ipython | sed '/^$/N;/^\\n$/D'";
var command = '!' + nbconvert_command + ' | ' + grep_command + ' > ' + script_file;

if (Jupyter.notebook.kernel) {
    Jupyter.notebook.kernel.execute(command);
}